# Week 2: Random Walk & Gambler's Ruin Problem

**Objective:** Understand the Random Walk as a fundamental stochastic process and analyze one of its most famous applications, the Gambler's Ruin problem.

## Part 1: The Random Walk

### Step 1: Build Intuition

Imagine a person walking along a straight line. They start at position zero. Every second, they flip a coin. If it's heads, they take one step to the right (+1). If it's tails, they take one step to the left (-1). Their path is unpredictable, yet it's built from simple, random steps. This is the essence of a **random walk**.

This simple model can describe many real-world phenomena:
- The movement of a stock price (up or down).
- The path of a molecule in a gas.
- A foraging animal's search for food.

### Step 2: Understand the Core Idea

A random walk is a path built by summing up a sequence of random steps.

- **Position at time `n`:** This is the sum of all the random steps taken up to that point, plus the starting position.
- **Random Step:** Each step is a random variable. In the simplest case, the steps are independent and identically distributed (i.i.d.).

The key idea is that the *next position only depends on the current position and the next random step*. This is a precursor to the Markov property we will see in Week 3.

### Step 3: Learn the Definitions and Formulas

**Definition: Simple Symmetric Random Walk**

A simple 1D random walk is a discrete-time stochastic process \(S_n\) defined by:

$$ S_n = S_0 + \sum_{i=1}^{n} X_i $$

Where:
- \(S_n\) is the position at time step \(n\).
- \(S_0\) is the starting position (often 0).
- \(X_i\) are independent and identically distributed (i.i.d.) random variables representing the steps.

For a **simple symmetric** random walk, the steps \(X_i\) are:

$$ X_i = \begin{cases} +1 & \text{with probability } p = 0.5 \\ -1 & \text{with probability } q = 1-p = 0.5 \end{cases} $$

### Step 4: Apply and Practice

Let's simulate a few random walks to see how different they can look, even when they come from the exact same process. This highlights the "randomness" inherent in a stochastic process. A single realization (one simulated path) is just one of infinitely many possibilities.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set a style for plots for better visualization
plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
def simulate_random_walk(num_steps, start_pos=0):
    """Simulates a 1D simple symmetric random walk."""
    # Steps are +1 or -1 with equal probability
    steps = np.random.choice([-1, 1], size=num_steps)
    # The first position is the starting position, so we prepend it
    # and then take the cumulative sum of steps.
    positions = np.concatenate(([start_pos], steps)).cumsum()
    return positions

# Simulation parameters
N_STEPS = 200
N_WALKS = 5 # Number of different walks to simulate
time_steps = np.arange(N_STEPS + 1)

# Plot the results
plt.figure(figsize=(14, 8))

for i in range(N_WALKS):
    # Run a new simulation for each walk
    walk_path = simulate_random_walk(N_STEPS)
    plt.step(time_steps, walk_path, where='mid', label=f'Walk {i+1}')

plt.title(f'{N_WALKS} Simulations of a Simple Random Walk')
plt.xlabel('Time Steps')
plt.ylabel('Position')
plt.axhline(0, color='black', linestyle='--', label='Start Position')
plt.legend()
plt.show()

---

## Part 2: The Gambler's Ruin Problem

### Step 1: Build Intuition

The Gambler's Ruin is a classic application of the random walk. Imagine a gambler starting with some money. They play a series of games, winning or losing $1 in each round. They have two goals:
1.  A **winning goal**: a target amount of money they want to reach.
2.  A **losing limit**: $0, at which point they are "ruined" and must stop playing.

The gambler's fortune is a random walk, but with a twist: there are two **absorbing barriers**. Once the gambler's fortune hits either 0 or their target, the game ends. The key question is: what is the probability that the gambler goes broke before reaching their goal?

### Step 2: Understand the Core Idea

This problem introduces the concept of **absorbing states** to a random walk.

- **State:** The gambler's current capital (e.g., \$k).
- **State Space:** The possible amounts of money the gambler can have, from \$0 to \$N (the target). So, the states are \({0, 1, 2, ..., N}\).
- **Transitions:** In each step, the state can change from \(k\) to \(k+1\) (win) or \(k-1\) (loss).
- **Absorbing Barriers:** If the state becomes 0 or N, it stays there forever. The random walk stops.

We want to find the probability of being absorbed at the 0 barrier, given a starting capital \(k\).

### Step 3: Learn the Definitions and Formulas

Let:
- \(k\) = The gambler's initial capital.
- \(N\) = The target capital (the house's capital or winning goal).
- \(p\) = The probability of winning \$1 in a single game.
- \(q = 1-p\) = The probability of losing \$1 in a single game.
- \(P_k\) = The probability of eventual ruin (reaching 0) given a starting capital of \(k\).

The probability of ruin, \(P_k\), can be calculated with the following formulas:

1.  **For a biased game (\(p \neq 0.5\)):**
$$ P_k = \frac{(q/p)^k - (q/p)^N}{1 - (q/p)^N} $$

2.  **For a fair game (\(p = 0.5\)):**
$$ P_k = 1 - \frac{k}{N} $$

### Step 4: Apply and Practice

Let's write a function to calculate the probability of ruin and then visualize how this probability changes based on the starting capital and the fairness of the game. This is where the formula becomes intuitive.

**Scenario:** A gambler starts with some money and wants to reach a total of \$100. What is their probability of going broke?

In [ ]:
def probability_of_ruin(start_capital, target_capital, win_prob):
    """
    Calculates the probability of a gambler going broke.
    
    Args:
        start_capital (int): The initial amount of money (k).
        target_capital (int): The winning goal (N).
        win_prob (float): The probability of winning a single round (p).
        
    Returns:
        float: The probability of ruin.
    """
    # Handle the edge case of a fair game (p=0.5)
    if win_prob == 0.5:
        return 1 - (start_capital / target_capital)
    
    # Handle the biased game case (p != 0.5)
    q = 1 - win_prob
    ratio = q / win_prob
    
    numerator = (ratio ** start_capital) - (ratio ** target_capital)
    denominator = 1 - (ratio ** target_capital)
    
    return numerator / denominator

# --- Parameters ---
TARGET = 100 # The gambler's goal is $100

# Let's test with a specific starting point
start = 50
p_fair = 0.5
p_unfair = 0.49 # A game slightly biased against the gambler

ruin_prob_fair = probability_of_ruin(start, TARGET, p_fair)
ruin_prob_unfair = probability_of_ruin(start, TARGET, p_unfair)

print(f"Starting with ${start} and aiming for ${TARGET}:")
print(f"- In a fair game (p=0.5), probability of ruin is: {ruin_prob_fair:.2%}")
print(f"- In an unfair game (p=0.49), probability of ruin is: {ruin_prob_unfair:.2%}")

Notice how even a tiny 1% disadvantage (`p=0.49` instead of `p=0.5`) dramatically increases the probability of ruin! Let's visualize this effect across all possible starting amounts.

In [ ]:
# Generate a range of starting capitals from $1 to $99
k_values = np.arange(1, TARGET)

# Calculate ruin probabilities for each starting capital
ruin_fair_game = [probability_of_ruin(k, TARGET, p_fair) for k in k_values]
ruin_unfair_game = [probability_of_ruin(k, TARGET, p_unfair) for k in k_values]
ruin_favorable_game = [probability_of_ruin(k, TARGET, 0.51) for k in k_values]

# Plot the results
plt.figure(figsize=(14, 8))

plt.plot(k_values, ruin_fair_game, label='Fair Game (p=0.5)', linestyle='--')
plt.plot(k_values, ruin_unfair_game, label='Unfavorable Game (p=0.49)', color='red')
plt.plot(k_values, ruin_favorable_game, label='Favorable Game (p=0.51)', color='green')

plt.title('Gambler\'s Ruin: Probability of Ruin vs. Starting Capital')
plt.xlabel('Starting Capital ($k$)')
plt.ylabel('Probability of Ruin')
plt.xlim(0, TARGET)
plt.ylim(0, 1)
plt.legend()
plt.grid(True, which='both', linestyle='-', linewidth=0.5)
plt.show()

**Interpretation of the Plot:**
- **Fair Game (Blue Dashed Line):** The probability of ruin decreases linearly. If you start with \$50 out of \$100, you have a 50% chance of ruin. If you start with \$90, you have only a 10% chance.
- **Unfavorable Game (Red Line):** The curve is bowed upwards. Even with a tiny disadvantage, the probability of ruin is significantly higher than in a fair game, especially if you start with a smaller amount of capital. Notice that even starting with \$80 gives you a ~40% chance of ruin!
- **Favorable Game (Green Line):** The curve is bowed downwards. A small advantage drastically reduces your chance of ruin.

## Summary & Next Steps

In this notebook, we've explored:
1.  The **Random Walk** as a sum of i.i.d. random steps.
2.  How to simulate and visualize random walks to understand their inherent variability.
3.  The **Gambler's Ruin Problem** as a random walk with absorbing barriers.
4.  How to use the formal equations to calculate the probability of ruin and see how sensitive it is to even a small bias in the game's fairness.

The Random Walk is a fundamental example of a **Markov Chain**, which is our topic for **Weeks 3-4**. A process has the Markov property if the future is independent of the past, given the present. In our random walk, to know the next position, all we need is the current position—we don't need to know how we got there. We will formalize this powerful idea next.